# **Part 1 - Google Merchant Store**

1. Get access to dataset https://developers.google.com/analytics/bigquery/web-ecommerce-demo-dataset
2. Please provide SQL queries that solve for the following:
    1. How many sessions does each user create?
    2. How much time does it take on average to purchase an item? How this time is distributed across users/items?
    3. What’s the overall conversion rate for transactions by device.category and device.mobile_brand_name?
    4. What is the most popular item for organic traffic for desktop platform?
3. Please create a dashboard that allows users to view the most popular items by traffic source and platform
4. Please provide any insight from data, that seems most interesting to you


In [ ]:
# @title Setup
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

project = 'focus-electron-392123' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

In [ ]:
data_table.disable_dataframe_formatter()

**1. How many sessions does each user create?**

In [ ]:
job = client.get_job('bquxjob_1abd935d_1893cfca256') # Job ID inserted based on the query results selected to explore
print(job.query)

SELECT sessions_cnt, COUNT(user_pseudo_id) AS users
FROM (
  SELECT 
    user_pseudo_id,
    COUNT(param.value.int_value) AS sessions_cnt
  FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_20210131*`,
  UNNEST(event_params) AS param
  WHERE 
    event_name = 'session_start'
    AND param.key IN ('ga_session_id')
  GROUP BY user_pseudo_id
  )t
GROUP BY sessions_cnt
ORDER BY users DESC


In [ ]:
# Distribution by session's quantity
res = job.to_dataframe()
res

,sessions_cnt,users
0,1,2241
1,2,189
2,3,35
3,4,6
4,7,1
5,5,1


In [ ]:
job1 = client.get_job('bquxjob_13685629_1893cfd9449')
print(job1.query)

  SELECT 
    COUNT(param.value.int_value) AS id_session
  FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_20210131*`,
  UNNEST(event_params) AS param
  WHERE event_name = 'session_start'
    AND param.key IN ('ga_session_id')


In [ ]:
# All sessions by users

res1 = job1.to_dataframe()
res1

,id_session
0,2760


**2. How much time does it take on average to purchase an item? How this time is distributed across users/items?**

I suppose, there is something wrong with total correctness of data, because:
1.   In purchase data there are **19 at all** purchases for particular item in the table
2.   These 19 purchases **have no item_id, no item_name and price**
3. Other suspitious cases:
some items **have different item_id, but the same item_name, price and actions with the same user** (for instance, this situation appears with this users: #5146751.2449405377, 2465265.7243532891)



In [ ]:
# Data for purchase event - I can't understand what was the items
job2 = client.get_job('bquxjob_13f50292_1893fc56d51')
print(job2.query)

select
  user_pseudo_id,
  event_name,
  FORMAT_TIMESTAMP('%Y-%m-%d %H:%M', TIMESTAMP_MICROS(event_timestamp)) AS event_date,
  item_id,
  item_name,
  price
FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_20210131*`,
UNNEST(items) AS item
WHERE event_name = 'purchase' 
--  AND item_id is not null 
--  AND item_name is not null
ORDER BY user_pseudo_id, item_id


In [ ]:
# Here I show you only 5 items, but other looks the same
res2 = job2.to_dataframe()
res2[:5]

,user_pseudo_id,event_name,event_date,item_id,item_name,price
0,1617434.1535145542,purchase,2021-01-31 07:54,(not set),(not set),NaN
1,1617434.1535145542,purchase,2021-01-31 08:11,(not set),(not set),NaN
2,21683663.4440188892,purchase,2021-01-31 09:53,(not set),(not set),NaN
3,21818790.8215652903,purchase,2021-01-31 02:28,(not set),(not set),NaN
4,2465265.7243532891,purchase,2021-01-31 01:38,(not set),(not set),NaN


In [ ]:
# Anyway I decided to explore client's funnel

In [ ]:
job3 = client.get_job('bquxjob_1dad3e4_1893fd33ceb')
print(job3.query)

SELECT t.user_pseudo_id, item_id, first_step, view_item, begin_checkout, purchase_item
FROM (
SELECT 
  user_pseudo_id,
  item_id,
  MIN(CASE WHEN event_name = 'view_item' THEN TIMESTAMP_MICROS(event_timestamp) END) view_item,
  MIN(CASE WHEN event_name = 'begin_checkout' THEN TIMESTAMP_MICROS(event_timestamp) END) begin_checkout,
  MIN(CASE WHEN event_name = 'purchase' THEN TIMESTAMP_MICROS(event_timestamp) END) purchase_item
FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_20210131*`,
UNNEST(items) AS item
GROUP BY user_pseudo_id, item_id
) t
LEFT JOIN 
(
  select 
  user_pseudo_id,
  MIN(TIMESTAMP_MICROS(event_timestamp)) as first_step
FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_20210131*`
WHERE event_name IN ('page_view', 'session_start', 'first_visit') # I assume, that these are the firt step in user journey on the website
GROUP BY user_pseudo_id 
) t2 ON t.user_pseudo_id = t2.user_pseudo_id
ORDER BY t.user_pseudo_id


In [ ]:
res3 = job3.to_dataframe()
res3

,user_pseudo_id,item_id,first_step,view_item,begin_checkout,purchase_item
0,10048419.1830672374,(not set),2021-01-31 07:50:46.451188+00:00,NaT,NaT,NaT
1,1026454.4271112504,(not set),2021-01-31 05:05:10.766593+00:00,NaT,NaT,NaT
2,10309845.4362790904,(not set),2021-01-31 11:16:08.360514+00:00,NaT,NaT,NaT
3,1037360.4939379761,(not set),2021-01-31 08:52:31.368899+00:00,NaT,NaT,NaT
4,1037755.3882512534,(not set),2021-01-31 10:58:54.295924+00:00,NaT,NaT,NaT
...,...,...,...,...,...,...
6734,9470937.3851692393,GGOEAFBA115599,2021-01-31 23:56:44.696885+00:00,2021-01-31 23:59:01.915345+00:00,NaT,NaT
6735,9470937.3851692393,GGOEAOCB092799,2021-01-31 23:56:44.696885+00:00,2021-01-31 23:59:01.915345+00:00,NaT,NaT
6736,97296388.9677363066,(not set),2021-01-31 16:04:03.537004+00:00,NaT,NaT,NaT
6737,9759528.0012004676,(not set),2021-01-31 15:05:07.712314+00:00,NaT,NaT,NaT


In [ ]:
job4 = client.get_job('bquxjob_39adc89a_189404a8537')
print(job4.query)

SELECT 
  COUNT(distinct t.user_pseudo_id) As unique_users,
  COUNT(item_id) As items_cnt,
  COUNT(first_step) As first_step_cnt,
  COUNT(view_item) As view_item_cnt, 
  COUNT(begin_checkout) As begin_checkout_cnt,
  COUNT(purchase_item) As purchase_item_cnt
FROM (
SELECT 
  user_pseudo_id,
  item_id,
  MIN(CASE WHEN event_name = 'view_item' THEN TIMESTAMP_MICROS(event_timestamp) END) view_item,
  MIN(CASE WHEN event_name = 'begin_checkout' THEN TIMESTAMP_MICROS(event_timestamp) END) begin_checkout,
  MIN(CASE WHEN event_name = 'purchase' THEN TIMESTAMP_MICROS(event_timestamp) END) purchase_item
FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_20210131*`,
UNNEST(items) AS item
GROUP BY user_pseudo_id, item_id
) t
LEFT JOIN 
(
  select 
  user_pseudo_id,
  MIN(TIMESTAMP_MICROS(event_timestamp)) as first_step
FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_20210131*`
WHERE event_name IN ('page_view', 'session_start', 'first_visit') # I assume, that thes

In [ ]:
res4 = job4.to_dataframe()
res4

,unique_users,items_cnt,first_step_cnt,view_item_cnt,begin_checkout_cnt,purchase_item_cnt
0,840,6739,6739,5939,63,17


In [ ]:
# Считаю среднее время по событиям
job5 = client.get_job('bquxjob_58d4afb7_189404295d3')
print(job5.query)

SELECT 
  time(timestamp_seconds(cast(avg(begin_checkout_first_step_diff) As int64))) As avg_begin_checkout_first_step,
  time(timestamp_seconds(cast(avg(begin_checkout_view_item_diff) As Int64))) As avg_begin_checkout_view_item_diff,
  time(timestamp_seconds(cast(avg(purchase_item_first_step_diff) As Int64))) As avg_purchase_item_first_step_diff,
  time(timestamp_seconds(cast(avg(purchase_item_view_item_diff)As Int64))) AS avg_purchase_item_view_item_diff,
  time(timestamp_seconds(cast(avg(purchase_begin_checkout_diff)As Int64))) AS avg_purchase_begin_checkout_diff
FROM (
SELECT 
  t.user_pseudo_id, 
  item_id, 
  TIMESTAMP_DIFF(begin_checkout,first_step, SECOND) As begin_checkout_first_step_diff,
  TIMESTAMP_DIFF(begin_checkout,view_item, SECOND) As begin_checkout_view_item_diff,
  TIMESTAMP_DIFF(purchase_item, first_step,SECOND) As purchase_item_first_step_diff,
  TIMESTAMP_DIFF(purchase_item, view_item, SECOND) As purchase_item_view_item_diff,
  TIMESTAMP_DIFF(purchase_item, begin_

In [ ]:
# Average time from session_start to begging_checkout - 1 hour 17 minutes
# Average time from session_start before the purchase  - 19 minutes
# The numbers look inconsistent, I suppose, because of incorrect values in @purchase field

# Average time from  begging_checkout to the purchase - 3 minutes
res5 = job5.to_dataframe()
res5

,avg_begin_checkout_first_step,avg_begin_checkout_view_item_diff,avg_purchase_item_first_step_diff,avg_purchase_item_view_item_diff,avg_purchase_begin_checkout_diff
0,01:17:51,NaT,00:19:16,NaT,00:03:27


**3. What’s the overall conversion rate for transactions by device.category and device.mobile_brand_name?**


Converstion rate is **transactions divided by sessions**, but
when I explored ecommerce.transaction_id field in the table --  results were not satisfied.
**Transaction_id consists of null and not_set**

In [ ]:
job6 = client.get_job('bquxjob_12350d4b_18940bdd2cb')
print(job6.query)

res6 = job6.to_dataframe()
res6

select distinct ecommerce.transaction_id
from `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_20210131*`


,transaction_id
0,None
1,(not set)


**I decided to explore the situation through session and attempts of purchuses**

In [ ]:
#Distibution by CR of mobile_brand_name

job7 = client.get_job('bquxjob_3357d598_18940af0ae7')
print(job7.query)

res7 = job7.to_dataframe()
res7

SELECT t.*, 
ROUND((begin_checkout / first_step) * 100, 2) AS CR_checkout_first_step
--ROUND((purchase_item / begin_checkout) * 100, 2) AS CR_purchase_checkout
FROM(
SELECT 
  device.mobile_brand_name,
  SUM(CASE WHEN event_name IN ('page_view', 'session_start', 'first_visit') THEN 1 ELSE 0 END) first_step,
  SUM(CASE WHEN event_name = 'begin_checkout' THEN 1 ELSE 0 END) begin_checkout,
  SUM(CASE WHEN event_name = 'purchase' THEN 1 ELSE 0 END) purchase_item
FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_20210131*`
GROUP BY device.mobile_brand_name
)t
ORDER BY CR_checkout_first_step DESC




,mobile_brand_name,first_step,begin_checkout,purchase_item,CR_checkout_first_step
0,<Other>,1132,31,2,2.74
1,Huawei,178,4,3,2.25
2,Google,4839,88,9,1.82
3,Microsoft,393,7,1,1.78
4,Samsung,1097,18,0,1.64
5,Apple,6211,82,3,1.32
6,Xiaomi,333,4,1,1.20
7,Mozilla,202,0,0,0.00


In [ ]:
#Distibution by CR of mobile_category

job8 = client.get_job('bquxjob_46bf978_18940ad5613')
print(job8.query)

res8 = job8.to_dataframe()
res8

SELECT t.*, 
ROUND((begin_checkout / first_step) * 100, 2) AS CR_checkout_first_step,
ROUND((purchase_item / begin_checkout) * 100, 2) AS CR_purchase_checkout
FROM(
SELECT 
  device.category,
  SUM(CASE WHEN event_name IN ('page_view', 'session_start', 'first_visit') THEN 1 ELSE 0 END) first_step,
  SUM(CASE WHEN event_name = 'begin_checkout' THEN 1 ELSE 0 END) begin_checkout,
  SUM(CASE WHEN event_name = 'purchase' THEN 1 ELSE 0 END) purchase_item
FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_20210131*`
GROUP BY device.category
)t
ORDER BY first_step DESC


,category,first_step,begin_checkout,purchase_item,CR_checkout_first_step,CR_purchase_checkout
0,desktop,8230,129,11,1.57,8.53
1,mobile,5805,91,7,1.57,7.69
2,tablet,350,14,1,4.00,7.14


# **Part 2 - Python tasks**

**1. Please write an example of python code, that checks, if the string is palindrome**

- ”**abcba”** is palindrome.

- **”abca”** is not a palindrome.

In [ ]:
a = 'abcba'

def palindrome(s):

    if s == s[::-1]:
        print('YES')
    else:
        print('NO')


palindrome(a)

YES


In [ ]:
a = 'abcba'

def palindrome(s):

    i = 0
    j = len(s) - 1
    is_palindrom = True

    while i < j:
        if s[i] != s[j]:
            is_palindrom = False
        i += 1
        j -= 1
    if is_palindrom:
        print('YES')
    else:
        print('NO')


palindrome(a)

YES


**2. Please write an example of python code, that calculates the longest sequence of equal symbols.**

- ”**abbbcbba”** - the longest sequence is **b** in the length of 3.
- **”abbbccccaaa”** - the longest sequence is **c** in the length of 4.

In [ ]:
s = 'abbbccccaaa'

def longest(s):
    max_counter = 0
    max_char = None
    cnt = 1

    for i in range(len(s) - 1):
        if s[i] == s[i + 1]:
            cnt += 1
            if cnt > max_counter:
                max_counter = cnt
                max_char = s[i]
        else:
            cnt = 1

    return max_char

print(longest(s))

c
